In [ ]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

# Bước 1 :Lấy bias của các action

In [ ]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

# Bước 2: Ghép các feature với nhau 

In [ ]:
perx=[np.zeros((len(matrix_feature),getStateSize()))]
for i in range(0,len(matrix_feature)):
    for j in range(getStateSize()):
        perx[0][i][j] += matrix_feature[i][0][j]
        
# for i in range(0,getActionSize()):
#     for j in range(len(matrix_feature)):
#         perx[1][i][j] = 1
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1:
        matran2 += bias_greedy
        # print(matran2)
        action = np.argmax(matran2)
    else:
        action = np.argmax(matran2)
    return action,per

win, x = numba_main_2(p0, 10000,perx,0)

In [ ]:
print(win)
print(getActionSize())

9379
15
